<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Biophysical model

_by Oriana Chegwidden (CarbonPlan), October 28, 2020_

This notebook combines biophysical effects (e.g. albedo) from the literature
with the National Land Cover Dataset (NLCD) classification types to create a
layer of albedo effects due to forests. This answers the question: "How much
(roughly) do existing forests affect the climate by absorbing radiation?"


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from carbonplan_forests import load
import zarr
import os

First we load in the biophysical effects of trees on radiation (in units of [W
m-2]). These estimates were derived by digitizing Figure S3 in Anderson-Teixeira
et al (2012) DOI:10.1038/NCLIMATE1346. The second column includes the
corresponding NLCD classification for each of the biome regions in Figure S3.


In [ ]:
conversion = pd.read_csv(
    "../../carbonplan_forests/data/nlcd_albedo_conversions.csv"
)

Then we load in a sample NLCD classification type to use as a template so that
the albedo dataset we make is consistently-shaped with all the other datasets
we're using.


In [ ]:
nlcd_types_sample = load.nlcd(store="az", year=2016, classes=[41])
biophysical_effect = xr.zeros_like(nlcd_types_sample)

Each of the NLCD layers corresponds to a fractional coverage of each gridcell by
a given land cover classification. So, by multiplying each of the fractions by
the corresponding albedo effect we get the weighted portion of albedo effect
from that land cover type. When we then sum them all together we get an
aerally-averaged albedo impact of forests for each gridcell. So, for example, if
a gridcell has very little forest it will have a very low average albedo effect,
even if that type of forest has a high albedo effect. Because two of the biomes
in the albedo dataset corresponded to the same NLCD classification, we averaged
those two biophysical effects in the calculation.


In [ ]:
for classification in conversion["nlcd classification"].values:
    # If more than one effect corresponds to an NLCD classification
    # we use the mean of those effects.
    biphysical_effect_value = np.mean(
        conversion["biophysical effect (ecosystem - " "bare ground) [W m-2]"][
            conversion["nlcd classification"] == classification
        ]
    )
    biophysical_effect += (
        load.nlcd(store="az", year=2016, classes=[classification])
        * biphysical_effect_value
    )

We then mask out oceans (based upon the NCLD landcover), package it into a
dataset, and write it out as a zarr file to the CarbonPlan data bucket.


In [ ]:
nlcd_types_mask = load.nlcd(store="az", year=2016, classes="all")

ds = biophysical_effect.where(nlcd_types_mask).to_dataset(name="biophysical")

store2 = zarr.storage.ABSStore(
    "carbonplan-data",
    prefix="biophysical/biophysical.zarr",
    account_name="carbonplan",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
)

ds.to_zarr(store2, consolidated=True, mode="w")